This is for scrapping houses data from Tayara.tn

we used beautiful soup to scrap the data , we scrapped both the data from the immoneuf and the immobilier categroies !

In [1]:
import requests
import csv
from bs4 import BeautifulSoup
import time


base_url = "https://www.tayara.tn/ads/c/{}/k/vendre/?page={}"
ss_Categories = ["Immobilier/Maisons%20et%20Villas", "Immobilier/Appartements"]
data = []
headers = ["Région", "Nature", "Superficie", "Nb_Salles De bain", "Nb_Chambres", "Prix", "Date_De_Modification", "Texte Annonce"]


for category in ss_Categories:
    for i in (1,5) :
        url = base_url.format(category,i)
        response = requests.get(url)
        if response.status_code == 200:
            print("Request successful!")
        else:
            print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")


Request successful!
Request successful!
Request successful!
Request successful!


In [2]:
from datetime import datetime, timedelta

def get_annonces_urls(soup):
    """Extracts all ad URLs from a parent container holding multiple 'mx-0' divs."""
    urls = []
    try:
        
        # Find all child divs with class "mx-0"
        annonces = soup.find_all("article", class_="mx-0")

        
        for annonce in annonces:
            try:
                link_tag = annonce.find("a")
                if link_tag:
                    url = link_tag.get("href")
                    if url and not url.startswith("http"):
                        full_url = "https://www.tayara.tn" + url
                    else:
                        full_url = url
                    if full_url and full_url not in urls:
                        urls.append(full_url)
            except Exception as e:
                print(f"Error extracting URL from annonce: {e}")
    except Exception as e:
        print(f"Error finding parent container or annonces: {e}")
    return urls

def get_annonce_data(url):
    """Extracts data for a specific ad URL."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {url}: {e}")
        return
    
    soup = BeautifulSoup(response.text, "html.parser")

    prix = region = date_posted = nature = superficie = nb_salles_de_bain = nb_chambres = text_annonce = "N/A"


    
    try:
        data_tag = soup.find('data', class_='font-bold font-arabic text-red-600 text-2xl')
        if data_tag:
            prix = data_tag.get("value")
        
    except:
        pass

    try:
        region_time = soup.find("div", class_="flex items-center space-x-2 mb-1").text.strip()
        region = region_time.split(",")[0].strip() if "," in region_time else "N/A"
        time_posted = region_time.split(",")[1].strip() if "," in region_time else "N/A"
        # Calculate the date posted based on the time_posted value
        if "hour" in time_posted:
            hours_ago = int(time_posted.split()[0])
            date_posted = (datetime.now() - timedelta(hours=hours_ago)).strftime('%Y-%m-%d')
        elif "day" in time_posted:
            days_ago = int(time_posted.split()[0])
            date_posted = (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
        else:
            date_posted = "N/A"
    except:
        pass

    try:
        na = soup.find("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80").text.strip()
        nature = na.split("Type de transaction")[1].strip()
    except:
        pass
    try:
        superficie = soup.find_all("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80")[1].text.split("Superficie")[1].strip()
    except:
        pass
    try:
        if "Salles de bains" in soup.find_all("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80")[2].text.strip():
            nb_salles_de_bain = soup.find_all("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80")[2].text.split("Salles de bains")[1].strip()
        elif "Chambres" in soup.find_all("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80")[2].text.strip():
            nb_chambres = soup.find_all("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80")[2].text.split("Chambres")[1].strip()
        else:
            nb_salles_de_bain = "N/A"

    except:
        pass

    try:
        if "Chambres" in soup.find_all("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80")[3].text.strip():
            nb_chambres = soup.find_all("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80")[3].text.split("Chambres")[1].strip()
        elif "Salles de bains" in soup.find_all("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80")[3].text.strip():
            nb_salles_de_bain = soup.find_all("div", class_="px-4 py-2 bg-gray-100/70 flex items-center rounded-md border border-gray-300/80")[3].text.split("Salles de bains")[1].strip()
    except:
        pass

    try:
        text_annonce = soup.find("p", class_="text-sm text-start text-gray-700 font-arabic whitespace-pre-line line-clamp-3").text.strip()
    except:
        pass

    print(f"Scraped data for {url}")
    data.append([region, nature, superficie, nb_salles_de_bain,nb_chambres, prix , date_posted, text_annonce])


def scrape_tayara():
    """Main function to iterate through pages and extract ad data."""
    for category in ss_Categories:
        print(f"\n--- Scraping category: {category} ---")
        
        for page in range(1, 50):  
            page_url = base_url.format(category, page)
            print(f"Scraping page: {page_url}")
            
            try:
                response = requests.get(page_url, timeout=10)
                response.raise_for_status()
            except requests.exceptions.RequestException as e:
                print(f"Failed to retrieve page {page_url}: {e}")
                continue
            
            soup = BeautifulSoup(response.text, "html.parser")
            annonce_urls = get_annonces_urls(soup)
            
            if not annonce_urls:
                print(f"No annonces found on page {page}. Skipping to next page.")
                continue

            for url in annonce_urls:
                get_annonce_data(url)
                
            # Delay to avoid being blocked
            time.sleep(2)
        
        print(f"Finished scraping category: {category}")
        
    print("\nScraping completed. Writing data to CSV file.")
    write_to_csv(data)

In [3]:
def write_to_csv(data):
    """Write the scraped data to a CSV file."""
    with open("tayara_v2.csv", "w", newline="", encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(data)
    print("Data successfully written to tayara.csv")

# Run the scraper
if __name__ == "__main__":
    scrape_tayara()


--- Scraping category: Immobilier/Maisons%20et%20Villas ---
Scraping page: https://www.tayara.tn/ads/c/Immobilier/Maisons%20et%20Villas/k/vendre/?page=1
Scraped data for https://www.tayara.tn/item/66e96193b3c14db342d0359a/Maisons%20et%20Villas/Ben_Arous/Rads/_VENDRE_IMMEUBLE_DE_RAPPORT_R3_RADES_PLAGE_/
Scraped data for https://www.tayara.tn/item/66e95f75b3c14db342d032a8/Maisons%20et%20Villas/Ben_Arous/Rads/_VENDRE_IMMEUBLE_R2_RADES_MELIANE_2/
Scraped data for https://www.tayara.tn/item/66e95de567b755ba9228c7a1/Maisons%20et%20Villas/Ben_Arous/Hammam_Lif/_VENDRE_IMMEUBLE_R2_EN_FLANC_DE_MONTAGNE_HAMMAMLIF/
Scraped data for https://www.tayara.tn/item/66bb31b1d0aad51c75f1b445/Maisons%20et%20Villas/Nabeul/Hammamet/A_vendre_une_belle_maison_S2_situe_aux_environs_de_Hammamet_5mn_en_voiture_de_la_ville_10mn_de_la_plage_/
Scraped data for https://www.tayara.tn/item/6761792bf39faeaa98bb7db6/Maisons%20et%20Villas/Ariana/La_Soukra/_a_vendre_une_villa_S3_avec_piscine_a_la_soukra_/
Scraped data for 